<a href="https://colab.research.google.com/github/Fuenfgeld/2022TeamADataEngineeringBC/blob/36-pr%C3%A4sentation-fortgeschrittene-transaktionen/Kopie_von_03_Transformationen_fuer_Fortgeschrittene.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Einlesen der Daten 

In [3]:
# Bibliotheken einmalig installieren

!pip install pyspark pandas pysqlite3 

     |████████████████████████████████| 281.4 MB 22 kB/s 
     |████████████████████████████████| 40 kB 4.8 MB/s 
     |████████████████████████████████| 198 kB 11.7 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=b0a7f8fb1a4c8a18b60de306308bf3bc4f93349f5fe04855d8f31a3533540281
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
  Created wheel for pysqlite3: filename=pysqlite3-0.4.7-cp37-cp37m-linux_x86_64.whl size=132867 sha256=520264f16bad29820bbc300a8980a6e0221e3d0987e9f39efb43023c636d2bd4
  Stored in directory: /root/.cache/pip/wheels/e5/bf/e1/b92ae6794ca15c7dcc2b1f8068ebb3f86e6affa5c0fe5e8f40
Successfully built pyspark pysqlite3


In [4]:
!wget -O create_data.py https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/main/Pr%C3%A4sentationen/02-Einfache%20Transformationen/create_data.py

--2022-05-09 22:03:09--  https://raw.githubusercontent.com/Fuenfgeld/2022TeamADataEngineeringBC/main/Pr%C3%A4sentationen/02-Einfache%20Transformationen/create_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 10138 (9.9K) [text/plain]
Saving to: ‘create_data.py’

create_data.py      100%[===================>]   9.90K  --.-KB/s    in 0s      

2022-05-09 22:03:09 (59.7 MB/s) - ‘create_data.py’ saved [10138/10138]



In [5]:
import sqlite3
import pandas as pd
import json
import os
import pyspark.sql.functions as func
from pyspark.sql.functions import col
os.system("python3 create_data.py")

0

In [6]:
## Datenbankdaten in Dataframe schreiben
connection_obj = sqlite3.connect('Diddly_Squat_Farm.db')
cursor_obj = connection_obj.cursor()

# Tabelle Fields ausgeben
for row in cursor_obj.execute('SELECT * FROM fields'):
        print(row)

# Tabelle Fields in Dataframe df_fields schreiben
df_fields = pd.read_sql_query("SELECT * FROM fields", connection_obj)
print('\n\nDies ist das erzeugte Dataframe:\n\n', df_fields)

connection_obj.close()

(5, 'Barn Ground', 10)
(6, 'Bank', 10)
(7, 'Far Brossler', 20)


Dies ist das erzeugte Dataframe:

    field_id    field_name  area_in_sqm
0         5   Barn Ground           10
1         6          Bank           10
2         7  Far Brossler           20


In [7]:
## (Py)Spark starten
from pyspark.sql import SparkSession
spark = SparkSession.builder.getOrCreate()

In [8]:
#DataFrame für Datenbank erzeugen
df_fields= spark.createDataFrame(df_fields) 
# df_fields.printSchema()
df_fields.show()

+--------+------------+-----------+
|field_id|  field_name|area_in_sqm|
+--------+------------+-----------+
|       5| Barn Ground|         10|
|       6|        Bank|         10|
|       7|Far Brossler|         20|
+--------+------------+-----------+



In [9]:
## JSON-/CSV-Dateien in Dataframes schreiben
df_fru = spark.read.options(header='True', multiline='True', inferSchema='True').csv('Fruits.csv')
df_veg = spark.read.option("multiline",True).json('Vegetables.json')

# erstellte Dataframes ausgeben (10 Einträge ausgeben)
df_fru.show(10)
df_veg.show(20)

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   1|               12|      0|            0|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|   7|               20|     60|           25|
+------------+-----+----+-----------------+-------+-------------+
only showing top 10 rows

+--------+-----+--------+-----------------+----+--

In [10]:
df_veg_correct_revenue = df_veg.withColumn("revenue", func.column("revenue.net"))
df_veg_correct_revenue.show(20)

df_fru_veg = df_fru.union(df_veg_correct_revenue)
df_fru_veg.show()

+--------+-----+-------+-----------------+----+-------------+
|    crop|field|revenue|water_consumption|week|yield_per_sqm|
+--------+-----+-------+-----------------+----+-------------+
|zucchini|    5|      0|                6|   1|            0|
|  radish|    6|      0|                4|   1|            0|
|zucchini|    5|      0|                5|   2|            0|
|  radish|    6|      0|                4|   2|            0|
|zucchini|    5|      0|                6|   3|            0|
|  radish|    6|      0|                5|   3|            0|
|zucchini|    5|      0|                7|   4|            0|
|  radish|    6|      0|                3|   4|            0|
|zucchini|    5|      0|                7|   5|            5|
|  radish|    6|      0|                6|   5|            0|
|zucchini|    5|      8|                9|   6|           10|
|  radish|    6|      0|                7|   1|            0|
|zucchini|    5|     16|               10|   7|           25|
|  radis

In [11]:
df_fru_veg = df_fru_veg.dropDuplicates()
df_fru_veg.show(20)

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|  11|             null|    150|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7|  10|               10|    100|           25|
|strawberries|    7|   4|               14|      0|            0|
|strawberries|    7|   1|               12|      0|            0|
|    zucchini|    5|   0|                7|      5|            5|
|      rad

# Transformationen für Fortgeschrittene

Damit Jeremy weitere Informationen aus den Daten filtern kann, muss er sich erneut einen Überblick verschaffen. 
+ Unterstützen Sie Jeremy, in dem Sie die Tabellen der Felder und anzubauenden Lebensmittels darstellen.

In [12]:
df_fields.show(10)
df_fru_veg.show(30)

+--------+------------+-----------+
|field_id|  field_name|area_in_sqm|
+--------+------------+-----------+
|       5| Barn Ground|         10|
|       6|        Bank|         10|
|       7|Far Brossler|         20|
+--------+------------+-----------+

+------------+-----+----+-----------------+-------+-------------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|
+------------+-----+----+-----------------+-------+-------------+
|strawberries|    7|   5|               14|      0|            5|
|strawberries|    7|   7|               20|     60|           25|
|strawberries|    7|   9|               24|    150|           25|
|strawberries|    7|   2|               10|      0|            0|
|strawberries|    7|   6|               18|     30|           10|
|strawberries|    7|   8|               26|    150|           25|
|strawberries|    7|  11|             null|    150|            0|
|strawberries|    7|   3|               12|      0|            0|
|strawberries|    7| 

#### Spark Functions 

Nutzen Sie Sparkfunktions um Jeremy den gesamte Wasserverbrauch im letzten Jahr darzustellen.




In [13]:
df_fru_veg.agg(func.sum("water_consumption")).show()

+----------------------+
|sum(water_consumption)|
+----------------------+
|                   285|
+----------------------+



Anschließend möchte Jeremy noch wissen, wie der Wasserverbrauch für die unterschiedlichen Gewächsarten war. Geben Sie hierfür den Wasserbedarf der Erdberren an.

In [14]:
df_fru_veg.filter(df_fru_veg.crop == "strawberries").agg(func.sum("water_consumption")).collect()

[Row(sum(water_consumption)=160)]

#### Groupby() Bedingung

Für die kommenden Wochen ist mit erhöhten Niederschlägen zu rechnen. Um zu wissen, wann es sich lohnt, die Felder zu bewässern, will sich Jeremy einen Überblick über die durchschnittliche Wassermenge pro Woche verschaffen.

+ Helfen Sie Jeremy  mittels groupby() Befehl dabei und visualisieren Sie den durchschnittlichen Wasserverbrauch.

In [15]:
df_alles_was_angebaut_wird = df_fru_veg.groupBy("week").agg(func.avg("water_consumption"))
df_alles_was_angebaut_wird.show()


+----+----------------------+
|week|avg(water_consumption)|
+----+----------------------+
|   0|     5.454545454545454|
|   7|                  20.0|
|  50|                   8.0|
|  32|                   8.0|
|   6|                  18.0|
|   9|                  24.0|
|   5|                  14.0|
|   1|                  12.0|
|  10|                  10.0|
|   3|                  12.0|
|   8|                  17.5|
|  11|                  null|
|   2|                  10.0|
|   4|                  14.0|
|  40|                  10.0|
|  16|                  10.0|
+----+----------------------+



Nachdem wir den Durchschnittlichen Wasserbrauch kennen, interessiert Jeremy wieviel Wasser jedes Lebensmittel pro Ernte verbraucht.  

In [16]:
gesamt_verbrauch =  df_fru_veg.groupBy("crop").agg(func.sum("water_consumption"))
gesamt_verbrauch.show()

+------------+----------------------+
|        crop|sum(water_consumption)|
+------------+----------------------+
|strawberries|                   160|
|    zucchini|                    80|
|      radish|                    45|
+------------+----------------------+



#### Join() Bedingung 

- Wie Sie in der Präsentation festgestellt haben, gibt es verschiedene Optionen zwei Tabellen miteinaner zu verbinden. 
+ In unserem Fall verwendenen wir nun den inner Join da wir alle Zeilen behalten möchten. 

Bisher musste Jereomy für die Übersicht der Felder und der einzelnen Lebensmittel immer zwei Tabellenen nutzen. 

+ Helfen Sie ihm nun die Arbeit mittels eines Joins zu erleichtern

In [17]:
gesamt_join = df_fru_veg.join(df_fields, df_fru_veg.field ==  df_fields.field_id,"inner")
gesamt_join.show(30)

+------------+-----+----+-----------------+-------+-------------+--------+------------+-----------+
|        crop|field|week|water_consumption|revenue|yield_per_sqm|field_id|  field_name|area_in_sqm|
+------------+-----+----+-----------------+-------+-------------+--------+------------+-----------+
|    zucchini|    5|   0|                7|      4|            0|       5| Barn Ground|         10|
|    zucchini|    5|   8|                9|      6|           10|       5| Barn Ground|         10|
|    zucchini|    5|  40|                5|     10|           25|       5| Barn Ground|         10|
|    zucchini|    5|  16|               10|      7|           25|       5| Barn Ground|         10|
|    zucchini|    5|  40|             null|     11|            0|       5| Barn Ground|         10|
|    zucchini|    5|  40|               13|      8|           25|       5| Barn Ground|         10|
|    zucchini|    5|  40|               12|      9|           25|       5| Barn Ground|         10|


Um die Wirtschaftlichkeit unseres Unternehmen zu prüfen, möchte Jeremy sich den gesamten Ertag des jeweiligen Nahrungsmittel anzeigen. 

+ Mulitplizieren Sie hierfür die Spalten yield_per_sqm und_sqm area_in_sqm
+ Erzeugen Sie eine neue Spalte im DataFrame für das Ergebnis
+ Tipp: hierfür können Sie die Funcktion **col** der Sparkfunctions verwenden.

In [18]:

gesamt_join = gesamt_join.withColumn("yield_per_area", func.col("yield_per_sqm") * func.col("area_in_sqm"))
gesamt_join.show()


+--------+-----+----+-----------------+-------+-------------+--------+-----------+-----------+--------------+
|    crop|field|week|water_consumption|revenue|yield_per_sqm|field_id| field_name|area_in_sqm|yield_per_area|
+--------+-----+----+-----------------+-------+-------------+--------+-----------+-----------+--------------+
|zucchini|    5|   0|                7|      4|            0|       5|Barn Ground|         10|             0|
|zucchini|    5|   8|                9|      6|           10|       5|Barn Ground|         10|           100|
|zucchini|    5|  40|                5|     10|           25|       5|Barn Ground|         10|           250|
|zucchini|    5|  16|               10|      7|           25|       5|Barn Ground|         10|           250|
|zucchini|    5|  40|             null|     11|            0|       5|Barn Ground|         10|             0|
|zucchini|    5|  40|               13|      8|           25|       5|Barn Ground|         10|           250|
|zucchini|

In [19]:

gesamt_ertrag =  gesamt_join.groupBy("crop").agg(func.sum("revenue"), func.sum("yield_per_area"), func.sum("water_consumption"))
gesamt_ertrag = gesamt_ertrag.select(col("crop").alias("Crop"),col("sum(revenue)").alias("revenue") , col("sum(water_consumption)").alias("water_consumption"), col("sum(yield_per_area)").alias("yield_per_area"))
gesamt_ertrag.show()


+------------+-------+-----------------+--------------+
|        Crop|revenue|water_consumption|yield_per_area|
+------------+-------+-----------------+--------------+
|    zucchini|     66|               80|          1150|
|strawberries|    640|              160|          2300|
|      radish|     32|               45|          4000|
+------------+-------+-----------------+--------------+



Nachdem wir den Ertrag für die einzelnen Gemüse- oder Obstsorten kennen, müssen wir Jeremy bei der Ermittelung des Gewinn je Lebensmittel unter die Arme greifen.

+ Hierfür benötigen wir die zuvor ermittelten Liter je Sorte welche wir mit dem Wert von 0.2 Cent verrechnen. 
 
+ Anschließend müssen noch die Erwerbskosten für die  Pflanzensamen miteinbeziehen um unsere Produktionskosten zu erhalten.
  >Kosten für Samen :
  + Erdbeeren = 50 Cent
  + Rettich = 10 Cent
  + Zucchini = 15 Cent 


+ Geben Sie das Ergebnis absteigend an. 

Zunächst müssen wir noch bei dem Dataframe noch eine Spalte mit den Kosten für die Saatkosten hinzufügen.

In [20]:
from pyspark.sql.window import Window as W
rating = [15,50,10]
b = spark.createDataFrame([(l,) for l in rating], ['cost_of_seed'])
gesamt_ertrag = gesamt_ertrag.withColumn("idx", func.monotonically_increasing_id())
b = b.withColumn("idx", func.monotonically_increasing_id())

windowSpec = W.orderBy("idx")
gesamt_ertrag = gesamt_ertrag.withColumn("idx", func.row_number().over(windowSpec))
b = b.withColumn("idx", func.row_number().over(windowSpec))


gesamt_ertrag = gesamt_ertrag.join(b, gesamt_ertrag.idx == b.idx).drop("idx")
gesamt_ertrag.show()

+------------+-------+-----------------+--------------+------------+
|        Crop|revenue|water_consumption|yield_per_area|cost_of_seed|
+------------+-------+-----------------+--------------+------------+
|    zucchini|     66|               80|          1150|          15|
|strawberries|    640|              160|          2300|          50|
|      radish|     32|               45|          4000|          10|
+------------+-------+-----------------+--------------+------------+



In [21]:
gesamt_ertrag = gesamt_ertrag.withColumn("revenue_after_water_cost",col("revenue") - (col("water_consumption")* 0.02))
gesamt_ertrag = gesamt_ertrag.withColumn("netto_revenue",col("revenue_after_water_cost") - col("cost_of_seed"))
gesamt_ertrag = gesamt_ertrag.withColumn("revenue_after_water_cost", func.round(gesamt_ertrag["revenue_after_water_cost"], 2)).withColumn("netto_revenue", func.round(gesamt_ertrag["netto_revenue"], 2))
gesamt_ertrag.show()

+------------+-------+-----------------+--------------+------------+------------------------+-------------+
|        Crop|revenue|water_consumption|yield_per_area|cost_of_seed|revenue_after_water_cost|netto_revenue|
+------------+-------+-----------------+--------------+------------+------------------------+-------------+
|    zucchini|     66|               80|          1150|          15|                    64.4|         49.4|
|strawberries|    640|              160|          2300|          50|                   636.8|        586.8|
|      radish|     32|               45|          4000|          10|                    31.1|         21.1|
+------------+-------+-----------------+--------------+------------+------------------------+-------------+



In [22]:
#revenue = Einnahme , yield = Ertrag
gesamt_ertrag.orderBy(col("netto_revenue").desc()).show()

+------------+-------+-----------------+--------------+------------+------------------------+-------------+
|        Crop|revenue|water_consumption|yield_per_area|cost_of_seed|revenue_after_water_cost|netto_revenue|
+------------+-------+-----------------+--------------+------------+------------------------+-------------+
|strawberries|    640|              160|          2300|          50|                   636.8|        586.8|
|    zucchini|     66|               80|          1150|          15|                    64.4|         49.4|
|      radish|     32|               45|          4000|          10|                    31.1|         21.1|
+------------+-------+-----------------+--------------+------------+------------------------+-------------+



#### Lambda Funtionen

Um unsere Lebensmittel in unserem Hofladen gewinnbringend vertreiben können, müssen wir Jeremy bei der Preisfindung unterstützen. 

+ Ermitteln Sie hierfür den Preis pro Stück und erzeugen hierfür eine neue Spalte in dem Dataframe


In [23]:
rdd2 = gesamt_ertrag.rdd.map(lambda x: (x["Crop"], x["netto_revenue"]*1.19*0.01, x["netto_revenue"]*1.59*0.01)) 
cost_overview = rdd2.toDF(["Crop","min_selling_price_per_piece", "lucrative_selling_price"])
cost_overview = cost_overview.withColumn("min_selling_price_per_piece", func.round(cost_overview["min_selling_price_per_piece"], 2)).withColumn("lucrative_selling_price", func.round(cost_overview["lucrative_selling_price"], 2))
cost_overview.withColumn("currency", func.lit("£")).orderBy(col("lucrative_selling_price").desc()).show()

+------------+---------------------------+-----------------------+--------+
|        Crop|min_selling_price_per_piece|lucrative_selling_price|currency|
+------------+---------------------------+-----------------------+--------+
|strawberries|                       6.98|                   9.33|       £|
|    zucchini|                       0.59|                   0.79|       £|
|      radish|                       0.25|                   0.34|       £|
+------------+---------------------------+-----------------------+--------+



Für die Nutzung von Lambda können Sie auch ganze Funktionen aufgerufen werden. Dies wird am folgenden Beispiel erklärt. 

In [24]:
# By Calling function
def calc_func(x):
    Crop = x.Crop
    Netto_Revenue = x.netto_revenue 
    min_selling_price_per_piece = Netto_Revenue*1.19*0.01
    lucrative_selling_price = Netto_Revenue*1.59*0.01
    return (Crop, min_selling_price_per_piece, lucrative_selling_price)

rdd_process=gesamt_ertrag.rdd.map(lambda x: calc_func(x))
cost_overview_with_func = rdd_process.toDF(["Crop","min_selling_price_per_piece", "lucrative_selling_price"])
cost_overview_with_func = cost_overview_with_func.withColumn("min_selling_price_per_piece", func.round(cost_overview_with_func["min_selling_price_per_piece"], 2)).withColumn("lucrative_selling_price", func.round(cost_overview_with_func["lucrative_selling_price"], 2))
cost_overview_with_func.withColumn("currency", func.lit("£")).orderBy(col("lucrative_selling_price").desc()).show()


+------------+---------------------------+-----------------------+--------+
|        Crop|min_selling_price_per_piece|lucrative_selling_price|currency|
+------------+---------------------------+-----------------------+--------+
|strawberries|                       6.98|                   9.33|       £|
|    zucchini|                       0.59|                   0.79|       £|
|      radish|                       0.25|                   0.34|       £|
+------------+---------------------------+-----------------------+--------+



####Übung


*   Verwenden Sie erneut die Corn.json welche die Daten für das Getreide enthält.
*   Ermitteln Sie den Wasserverbrauch für die jeweilige Getreidesorte.
+   Ermitteln Sie die Kosten für den Getreideanbau und den Verkaufspreis  

